In [14]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [15]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_pastis.ipynb


In [16]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [17]:
from scipy import spatial
from sklearn.decomposition import PCA

In [18]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [19]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [20]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [21]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [22]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,995.47,0.00,0.02,0.00,0.00,-0.00,0.00
3,0.00,299.98,-0.00,-0.00,-0.00,0.00,0.00
4,0.02,-0.00,75.55,-0.00,-0.00,-0.00,-0.00
5,0.00,-0.00,-0.00,14.57,-0.00,0.00,-0.00
6,0.00,-0.00,-0.00,-0.00,2.58,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,0.44,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.00,0.04


In [23]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00007,0.00002,0.00005,-0.00005,0.00006
3,0.00000,1.00000,-0.00002,-0.00005,-0.00004,0.00001,0.00001
4,0.00007,-0.00002,1.00000,-0.00003,-0.00003,-0.00004,-0.00001
5,0.00002,-0.00005,-0.00003,1.00000,-0.00002,0.00001,-0.00001
6,0.00005,-0.00004,-0.00003,-0.00002,1.00000,0.00002,0.00001
7,-0.00005,0.00001,-0.00004,0.00001,0.00002,1.00000,0.00005
8,0.00006,0.00001,-0.00001,-0.00001,0.00001,0.00005,1.00000


In [24]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.02671288951566483

In [25]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.41979286e+05 1.24998409e+04 7.60696884e+02 2.65538789e+01
 7.43256249e-01 1.61544052e-02 7.69626735e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998410,-0.054919,-0.012434,-0.002383,-0.000420,-0.000073,-0.000007
1,0.054141,0.997006,-0.054323,-0.009714,-0.001718,-0.000292,-0.000029
2,0.015209,0.052998,0.997162,-0.050526,-0.008553,-0.001457,-0.000148
3,0.003631,0.012091,0.049269,0.996780,-0.061173,-0.009993,-0.001013
4,0.000850,0.002834,0.011252,0.059540,0.994281,-0.087478,-0.008492
5,0.000219,0.000724,0.002879,0.014920,0.085390,0.990994,-0.102015
6,0.000044,0.000148,0.000588,0.003046,0.017182,0.100873,0.994746


In [26]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00158952, 0.00299429, 0.00283815, 0.00322011, 0.00571886,
       0.00900554, 0.00525394])